<a href="https://colab.research.google.com/github/Gana797/ML-model/blob/main/ricetypeclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od. download ("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ganaamruth
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 80.4MB/s]

In [18]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print(device)



cuda


In [19]:
data_df=pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [20]:
data_df.dropna(inplace=True)
data_df.drop(['id'], axis=1, inplace=True)
data_df.head()
print(data_df.shape)

(18185, 11)


In [21]:
print(data_df["Class"]. unique())

[1 0]


In [22]:
print(data_df["Class"]. value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [23]:
original_df=data_df.copy()

for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()



data_df.head()


,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(data_df.iloc[:, :-1])  # Extracting features (all columns except the last)
Y = np.array(data_df.iloc[:, -1])   # Extracting target (last column)

# Spliting dataset: 70% training, 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Spliting the test set into validation and test sets (each 15% of the original data)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)



X_train shape: (12729, 10)
X_val shape: (2728, 10)
X_test shape: (2728, 10)


In [25]:
class dataset(Dataset):
    def __init__(self, X, Y, device='cpu'):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [87]:
from torch.utils.data import DataLoader

training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=32, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size=32, shuffle=True)

In [88]:
for x, y in train_dataloader:
    print(x)
    print('======')
    print(y)
    break

tensor([[0.5656, 0.8046, 0.6144, 0.9712, 0.5389, 0.7521, 0.6509, 0.6474, 0.7401,
         0.7429],
        [0.8910, 0.8513, 0.9131, 0.9055, 0.8433, 0.9439, 0.9004, 0.7497, 0.8693,
         0.5289],
        [0.8921, 0.8775, 0.8807, 0.9226, 0.8440, 0.9445, 0.6578, 0.7509, 0.8676,
         0.5653],
        [0.9326, 0.9045, 0.8944, 0.9260, 0.8798, 0.9657, 0.8723, 0.7682, 0.8667,
         0.5737],
        [0.6463, 0.8588, 0.6590, 0.9706, 0.6173, 0.8039, 0.6943, 0.6973, 0.7290,
         0.7394],
        [0.5898, 0.8372, 0.6197, 0.9751, 0.5612, 0.7680, 0.7598, 0.6720, 0.7162,
         0.7664],
        [0.7089, 0.8091, 0.7753, 0.9330, 0.6804, 0.8420, 0.7456, 0.6913, 0.8135,
         0.5921],
        [0.7667, 0.8451, 0.7920, 0.9376, 0.7269, 0.8756, 0.6111, 0.7139, 0.8251,
         0.6054],
        [0.6648, 0.7668, 0.7610, 0.9252, 0.6350, 0.8154, 0.6554, 0.6715, 0.8086,
         0.5717],
        [0.5436, 0.7642, 0.6197, 0.9628, 0.5159, 0.7373, 0.9274, 0.6176, 0.7817,
         0.6996],
        [0

In [89]:
import torch
import torch.nn as nn

HIDDEN_NEURONS = 10

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
        self.linear = nn.Linear(HIDDEN_NEURONS, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

model = MyModel().to(device)


In [90]:
summary(model,(X.shape[1],) )

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [91]:
criterion=nn.BCELoss()
optimizer=Adam(model.parameters(),lr=1e-3)

In [92]:
import torch.optim as optim

optimizer = Adam(model.parameters(), lr=1e-3)




In [93]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []




In [94]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MyModel().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 10

for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val = 0
    total_loss_val = 0

    for data in train_dataloader:
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        prediction = model(inputs).squeeze(1)

        # Print shapes for debugging
        print( prediction.shape)


torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([32

In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for data in train_dataloader:
    inputs, labels = data

    inputs, labels = inputs.to(device), labels.to(device)

    # Forward pass
    prediction = model(inputs)

    print("Inputs Device:", inputs.device)
    print("Labels Device:", labels.device)
    print("Prediction Device:", prediction.device)
    print("Prediction Shape:", prediction.shape)

    break  # Exit after first batch for debugging


Inputs Device: cuda:0
Labels Device: cuda:0
Prediction Device: cuda:0
Prediction Shape: torch.Size([32, 1])


In [96]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to device
model = MyModel().to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 10

for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val = 0
    total_loss_val = 0

    for data in train_dataloader:
        # Correct variable name (inputs instead of input)
        inputs, labels = data

        # Move tensors to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        prediction = model(inputs)

        print("Prediction Shape Before Squeeze:", prediction.shape)

        if prediction.dim() > 1 and prediction.shape[1] == 1:
            prediction = prediction.squeeze(1)

        print("Prediction Shape After Squeeze:", prediction.shape)
        batch_loss = criterion(prediction, labels)
        total_loss_train += batch_loss.item()

        acc = (prediction.round() == labels).sum()

        print(acc)

        break
        break  # Exit after first batch



Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(22, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(21, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(19, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(19, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(17, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(16, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
tensor(11, device='cuda:0')
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch

In [97]:
for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    for i, data in enumerate(train_dataloader):  # Add enumerate to track iterations
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)

        prediction = model(inputs)

        print("Prediction Shape Before Squeeze:", prediction.shape)

        if prediction.dim() > 1 and prediction.shape[1] == 1:
            prediction = prediction.squeeze(1)

        print("Prediction Shape After Squeeze:", prediction.shape)

        batch_loss = criterion(prediction, labels)
        total_loss_train += batch_loss.item()

        acc = (prediction.round() == labels).sum().item()
        total_acc_train += acc
        total_loss_train_plot.append(round(total_loss_train / 1000, 4))
        total_loss_validation_plot.append(round(total_loss_val / 1000, 4))
        total_acc_train_plot.append(round(total_acc_train / len(training_data) * 100, 4))
        total_acc_validation_plot.append(round(total_acc_val / len(validation_data) * 100, 4))

        print(f"Epoch no. {epoch + 1} | Train Loss: {round(total_loss_train / 1000, 4)} | Train Accuracy: {round(total_acc_train / len(training_data) * 100, 4)}")
        print(f"Validation Loss: {round(total_loss_val / 1000, 4)} | Validation Accuracy: {round(total_acc_val / len(validation_data) * 100, 4)}")
        print("=" * 25)
        break# Exit after the first epoch
        break   # Exit after the first batch





Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
Epoch no. 1 | Train Loss: 0.0007 | Train Accuracy: 0.1336
Validation Loss: 0.0 | Validation Accuracy: 0.0
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
Epoch no. 2 | Train Loss: 0.0007 | Train Accuracy: 0.1336
Validation Loss: 0.0 | Validation Accuracy: 0.0
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
Epoch no. 3 | Train Loss: 0.0007 | Train Accuracy: 0.1414
Validation Loss: 0.0 | Validation Accuracy: 0.0
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
Epoch no. 4 | Train Loss: 0.0007 | Train Accuracy: 0.1414
Validation Loss: 0.0 | Validation Accuracy: 0.0
Prediction Shape Before Squeeze: torch.Size([32, 1])
Prediction Shape After Squeeze: torch.Size([32])
Epoch no. 5 | Train Loss: 0.0007 | Train Accuracy: 0.1257
Validati

In [98]:
acc= (prediction.round() == labels).sum().item()
print(acc)

11


In [101]:
# Testing Phase (No Gradients)
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0

    for inputs, labels in testing_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        prediction = model(inputs).squeeze(1)  # Ensure correct dimension

        # Compute loss
        batch_loss_test = criterion(prediction, labels).item()
        total_loss_test += batch_loss_test

        # Compute accuracy
        acc = (prediction.round() == labels).sum().item()
        total_acc_test += acc

print("Test Accuracy:", round(total_acc_test / len(testing_data) * 100, 4), "%")


Test Accuracy: 54.5455 %


In [110]:
area = 2353 / original_df['Area'].abs().max()
MajorAxisLength = 81 / original_df['MajorAxisLength'].abs().max()
MinorAxisLength = 42 / original_df['MinorAxisLength'].abs().max()
Eccentricity = 32 / original_df['Eccentricity'].abs().max()
ConvexArea = 12 / original_df['ConvexArea'].abs().max()
EquivDiameter = 33 / original_df['EquivDiameter'].abs().max()
Extent = 98 / original_df['Extent'].abs().max()
Perimeter = 927 / original_df['Perimeter'].abs().max()
Roundness = 677 / original_df['Roundness'].abs().max()
AspectRation = 24 / original_df['AspectRation'].abs().max()

my_prediction= model(torch.tensor([area, MajorAxisLength, MinorAxisLength, Eccentricity, ConvexArea, EquivDiameter, Extent, Perimeter, Roundness, AspectRation], dtype= torch.float32).to(device))


In [111]:
import torch

area = 2353 / original_df['Area'].abs().max()
MajorAxisLength = 81 / original_df['MajorAxisLength'].abs().max()
MinorAxisLength = 42 / original_df['MinorAxisLength'].abs().max()
Eccentricity = 32 / original_df['Eccentricity'].abs().max()
ConvexArea = 12 / original_df['ConvexArea'].abs().max()
EquivDiameter = 33 / original_df['EquivDiameter'].abs().max()
Extent = 98 / original_df['Extent'].abs().max()
Perimeter = 927 / original_df['Perimeter'].abs().max()
Roundness = 677 / original_df['Roundness'].abs().max()
AspectRation = 24 / original_df['AspectRation'].abs().max()


input_tensor = torch.tensor(
    [area, MajorAxisLength, MinorAxisLength, Eccentricity, ConvexArea, EquivDiameter,
     Extent, Perimeter, Roundness, AspectRation], dtype=torch.float32
).to(device)


input_tensor = input_tensor.unsqueeze(0)  # Shape becomes [1, 10]

#  prediction from the model
my_prediction = model(input_tensor)

# Print the output
print("Model Prediction:", my_prediction.item())  # Convert tensor to scalar


Model Prediction: 0.7625182271003723
